**WRITER'S IDENTITY**
- AZIMIL GANI ALAM
- Ph.D Student
- Dept. Energy & Process Tech - NTNU


**NOTE :**
- This Python File runs in GoogleColab
- This work uses
  - **Weather Data Record by WWW.SEKLIMA.NO**
  - **Outdoor Data Record by NILU Air Quality Monitoring Data**
- we want to filterize data results from CSV models become process-ready for Machine Learning process

**RULES**
1. Run Library Import
2. Select Notebook platform by run one of those commands.
3. Expand Schools Tab
 - Run Identity and Data Caller
 - Expand Classrooms
 - Run one of Klasserom- XXX
 - Run Processor. if done, select again one of Klasserom and run again the processor. it is cyclic process

to check the data cleaning result, run 'Data Preparation'.


# Libraries Import

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from datetime import datetime, timedelta
import pytz

In [5]:
#Folder naming directory
platform_directory = 'C:/Users/azimilga'   
platform_directory = 'G:/My Drive/Colab Notebooks'      
folder_raw     = 'data-raw'
folder_ready   = 'data-ready'
folder_year    = '2024'
folder_project = 'Project'

In [6]:
school_col_name   = 'school_id'
room_col_name     = 'room_id'
date_name         = 'date'
time_name         = 'time'
manuf_col_name    = 'manuf_id'
instr_col_name    = 'instr_id'
serial_col_name   = 'serial_id'
temp_set_name     = 'temp_o'
rel_hum_name      = 'rh_o'
sun_rad_name      = 'sun_o'
winds_name        =  'winds_o'
percipitation_name= 'rain_o'
pm25_name         = 'pm2.5_o'
pm10_name         = 'pm10_o'
pressure_name     = 'press_o'
gas_nox_name      = 'nox_o'
gas_no_name      = 'no_o'
gas_no2_name      = 'no2_o'

school_col_name   = 'school_id'
room_col_name     = 'room_id'
date_name         = 'date'
time_name         = 'time'
manuf_col_name    = 'manuf_id'
instr_col_name    = 'instr_id'
serial_col_name   = 'serial_id'

#identify a datetime name : wont be displayed. just coding purpose
datetime_name = 'datetime'

#filtering rows data regarding to desired specific time
start_time =  '00:00:00'
end_time   =  '23:59:59'
start_date =  '2024-01-01'
end_date   =  '2024-05-30'
date_format_show =  '%Y-%m-%d'
time_format_show =  '%H:%M'
spec_date = pd.to_datetime('today').strftime(date_format_show)
name_date = pd.to_datetime('today').strftime("%y%m%d")

# Define the timezones (UTC and Europe/Oslo)
utc_timezone = pytz.timezone('UTC')
oslo_timezone = pytz.timezone('Europe/Oslo')

# Volda

## Bratteberg

In [8]:
school_name = 'bratteberg'
manuf_id    = 'seklima'
instr_id    = 'volda lufthamn'
serial_id   = 'SN59680'
input_file_directory    = platform_directory + '/' + folder_project + '/' + folder_raw   + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'
export_file_directory   = platform_directory + '/' + folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'

weather_column_array = [  instr_col_name,serial_col_name, datetime_name, pressure_name, temp_set_name,rel_hum_name, winds_name,  percipitation_name]


## Øyra

In [20]:
school_name = 'øyra'
manuf_id    = 'seklima'
instr_id    = 'volda lufthamn'
serial_id   = 'SN59680'
input_file_directory  = platform_directory + '/' + folder_project + '/' + folder_raw + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'
export_file_directory   = platform_directory + '/'+ folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'

weather_column_array = [  instr_col_name,serial_col_name, datetime_name, pressure_name, temp_set_name,rel_hum_name, winds_name,  percipitation_name]

## Processing

In [9]:
folder_path1 = input_file_directory # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('.xlsx')] # get list of csv files
dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_excel(file_path  ) # read csv file into a dataframe delimiter=';', 
    dfs1.append(df) # append dataframe to list

# concatenate dataframes in the list into a single dataframe
DF1 = pd.concat(dfs1, ignore_index=True).iloc[:-1]
DF1.columns = weather_column_array
DF1[datetime_name] = pd.to_datetime((DF1[datetime_name]), format= '%d.%m.%Y %H:%M')
timezone = DF1[datetime_name].dt.tz_localize(oslo_timezone,  ambiguous='NaT', nonexistent='shift_forward').dt.tz_convert(utc_timezone)
DF1[datetime_name] =  timezone.dt.tz_localize(None)

#DF1.set_index(datetime_name, inplace=True)
DF1.iloc[:,3:] = DF1.iloc[:,3:].replace('-', regex=True).astype(float).interpolate(method='linear')  #to handle minus string values. tell pandas to read it as minus value, not a string typing

DF1[instr_col_name] = instr_id  
DF1[school_col_name] = school_name
DF1[serial_col_name] = serial_id   
DF1[manuf_col_name] = manuf_id
DF1[date_name] = DF1[datetime_name ].dt.strftime(date_format_show)
DF1[time_name] = DF1[datetime_name ].dt.strftime(time_format_show)

#we move columns named 'Date' and 'Time' to the front side
cols_to_move = [school_col_name, date_name, time_name, manuf_col_name, instr_col_name, serial_col_name ]
DF1         = DF1[ cols_to_move + [ col for col in DF1.columns if col not in cols_to_move ] ].drop([datetime_name], axis=1).dropna()

#put spesific Directory and File Name.
name_time = datetime.strptime((DF1[time_name].iloc[0]),'%H:%M').strftime('%H%M')
name_date = datetime.strptime((DF1[date_name].iloc[0]),'%Y-%m-%d').strftime('%Y%m%d')

directory_path = export_file_directory+'/'
filename = school_name+ '_'+ instr_id + '_'+serial_id +'_'+name_date +'_'+ name_time + '.csv'

# save dataframe (accumulative format) to CSV file in specified directory
DF1.to_csv(directory_path + filename, index= False)

# Oslo

## Brannfjell

In [23]:
school_name = 'brannfjell'
manuf_id    = 'seklima'
instr_id = 'Oslo - Blindern'
instr_id2   = ' & nilu manglerud'
serial_id = 'SN18700'
input_file_directory  = platform_directory + '/' + folder_project + '/' + folder_raw + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'

weather_column_array = [ instr_col_name,serial_col_name, datetime_name, temp_set_name, rel_hum_name, winds_name, percipitation_name, pressure_name, sun_rad_name]
pollution_column_array = [datetime_name,pm10_name,pm25_name]

## KubenVGS

In [125]:
school_name = 'kubenVGS'
manuf_id    = 'seklima'
instr_id = 'Oslo - Blindern'
instr_id2   = ' & nilu alnabru'
serial_id = 'SN18700'
input_file_directory  = platform_directory + '/' + folder_project + '/' + folder_raw + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'

weather_column_array = [ instr_col_name,serial_col_name, datetime_name, temp_set_name, rel_hum_name, winds_name, percipitation_name, pressure_name, sun_rad_name]
pollution_column_array = [datetime_name,pm10_name,pm25_name, gas_nox_name, gas_no2_name, gas_no_name]

## KubenYA

In [59]:
school_name = 'kubenFS'
manuf_id    = 'seklima'
instr_id = 'Oslo - Blindern'
instr_id2   = ' & nilu alnabru'
serial_id = 'SN18700'
input_file_directory  = platform_directory + '/' + folder_project + '/' + folder_raw + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'

weather_column_array = [ instr_col_name,serial_col_name, datetime_name, temp_set_name, rel_hum_name, winds_name, percipitation_name, pressure_name, sun_rad_name]
pollution_column_array = [datetime_name,pm10_name,pm25_name]

## Processing

In [ ]:
folder_path1 = input_file_directory # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('.xlsx')] # get list of csv files
dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_excel(file_path  ) # read csv file into a dataframe delimiter=';', 
    dfs1.append(df) # append dataframe to list

# concatenate dataframes in the list into a single dataframe
DF1 = pd.concat(dfs1, ignore_index=True).iloc[:-1]

DF1.columns = weather_column_array

DF1[datetime_name] = pd.to_datetime((DF1[datetime_name]), format= '%d.%m.%Y %H:%M')
timezone = DF1[datetime_name].dt.tz_localize(oslo_timezone,  ambiguous='NaT', nonexistent='shift_forward').dt.tz_convert(utc_timezone)
DF1[datetime_name] =  timezone.dt.tz_localize(None)

#special for Oslo
DF1.set_index(datetime_name, inplace=True)
# DFsolar= DF1[DF1[serial_col_name]!=serial_id][sun_rad_name].iloc[2:].reset_index().set_index(datetime_name).applymap(lambda x: max(0, x))
# DF1 =   DF1[DF1[serial_col_name]==serial_id].drop(sun_rad_name, axis=1).join(DFsolar, how='outer').dropna(axis=0).iloc[1:]
DF1.iloc[:,2:] = DF1.iloc[:,2:].replace('-', regex=True).astype(float).interpolate(method='linear')  #to handle minus string values. tell pandas to read it as minus value, not a string typing
DF1#.mean() #.info()


#importing data from NILU HISTORIKK DATA
manuf_id2    = 'niluaqmd'
input_file_directory2  = platform_directory + '/' + folder_project + '/' + folder_raw + '/' + folder_year + '/' + school_name + '/' + manuf_id2 + '/'
folder_path2 = input_file_directory2 # replace with the folder path
csv_files2 = [f for f in os.listdir(folder_path2) if f.endswith('.csv')] # get list of csv files
dfs1 = [] # list to store dataframes

for file in csv_files2:
    file_path = os.path.join(folder_path2, file) # get full path of csv file
    df = pd.read_csv(file_path,delimiter=";", header=3)#.iloc[5,:] # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list

# concatenate dataframes in the list into a single dataframe
DF2 = pd.concat(dfs1, ignore_index=True)
DF2 = DF2.drop(columns=DF2.filter(regex=f'Dekning', axis=1).columns, axis=1)
DF2.columns= pollution_column_array

DF2[datetime_name] = pd.to_datetime((DF2[datetime_name]), format= '%d.%m.%Y %H:%M')
timezone = DF2[datetime_name].dt.tz_localize(oslo_timezone,  ambiguous='NaT', nonexistent='shift_forward').dt.tz_convert(utc_timezone)
DF2[datetime_name] =  timezone.dt.tz_localize(None)

DF2 = DF2.set_index(datetime_name)
DF2 = DF2.astype(str).replace(',', '.', regex=True).astype(float)#.applymap(lambda x: max(0, x))#.astype(float)
DF2 = DF2.interpolate(method='linear').applymap(lambda x: max(0, x))

#JOINING SEKLIMA AND NILU HISTORIKK DATA
DF3 = DF1.join(DF2, how='outer').dropna()
DF3 = DF3.drop([instr_col_name,serial_col_name],axis=1).resample('1H').mean().reset_index()

DF3[instr_col_name] = instr_id  + instr_id2 
DF3[school_col_name] = school_name
DF3[serial_col_name] = serial_id   
DF3[manuf_col_name] = manuf_id + '+' + manuf_id2
DF3[date_name] = DF3[datetime_name ].dt.strftime(date_format_show)
DF3[time_name] = DF3[datetime_name ].dt.strftime(time_format_show)
DF3

#we move columns named 'Date' and 'Time' to the front side
cols_to_move = [school_col_name, date_name, time_name, manuf_col_name, instr_col_name, serial_col_name  ,   temp_set_name, rel_hum_name, winds_name  ]
DF3         = DF3[ cols_to_move + [ col for col in DF3.columns if col not in cols_to_move ] ].drop([datetime_name], axis=1).dropna()

DF3.iloc[:,6:]=DF3.iloc[:,6:].astype(float)#.describe()

#put spesific Directory and File Name.
name_time = datetime.strptime((DF3[time_name].iloc[0]),'%H:%M').strftime('%H%M')
name_date = datetime.strptime((DF3[date_name].iloc[0]),'%Y-%m-%d').strftime('%Y%m%d')

export_file_directory   = platform_directory + '/'+ folder_project + '/' + folder_ready + '/' + folder_year + '/'  + school_name + '/' + manuf_id + '/'
directory_path = export_file_directory+'/'
filename = school_name+ '_'+ manuf_id + '_'+serial_id +'_'+name_date +'_'+ name_time + '.csv'

# save dataframe (accumulative format) to CSV file in specified directory
DF3.to_csv(directory_path + filename, index= False)

In [ ]:
DF3